# JSON interface for diagram creation
------
## Example of Entity Relationship Diagram
- [ER-example with simple structure](#ER-example-with-simple-structure)
- [ER-second example with additional information](#ER-second-example-with-additional-information)

## Example of Flowchart Diagram
- [FC-example with simple structure](#FC-example-with-simple-structure)
- [Example of NTV data](#Example-of-NTV-data)
- [Example of NTV-TAB data](#Example-of-NTV-TAB-data)

-------
[Functions used to create diagram from json](https://github.com/loco-philippe/NTV/blob/main/json_ntv/json_mermaid.py)

## Example of Entity Relationship Diagram

### ER-example with simple structure

In [9]:
# The Json syntax is enriched by separator ':' or '::' in dict key to indicate that:
#  - the second part of the key string is the data type
#  - the dict value is a json entity (':') or a list of Json-NTV entities.
from base64 import b64encode
from IPython.display import Image, display
from json_ntv import Ntv, MermaidConnec

order_example = { 
    'json order example:$erDiagram' : { 
        'relationship': [ 
            [ 'CUSTOMER', 'exactly one', 'identifying', 'zero or more', 'ORDER',     'places'],
            [ 'ORDER',    'exactly one', 'identifying', 'one or more',  'LINE-ITEM', 'contains'] 
        ],
        'entity': {
            'CUSTOMER':  [ 
                ['string', 'name',         'PK', 'the name'], 
                ['string', 'custNumber'] 
            ], 
            'ORDER': [ 
                ['int',    'orderNumber',  'PK'],
                ['string', 'deliveryAdress'] 
            ],
            'LINE-ITEM': [ 
                ['string', 'productCode',  'PK'],
                ['int',    'quantity'],
                ['float',  'pricePerUnit'] 
            ]   
        },
     } }

diag = MermaidConnec.diagram(order_example)
display(Image(url="https://mermaid.ink/img/" + b64encode(diag.encode("ascii")).decode("ascii")))

### ER-second example with additional information

In [2]:
# in this example we use the potential of the NTV format to add additional information
# (without changing either the code or the output data)

order_example2 = { 
    'json order example 2:$erDiagram' : { 
        'relationship::': [ 
            [ {'1st entity':'CUSTOMER'}, 'exactly one', 'identifying', 'zero or more', {'2nd entity':'ORDER'}, 'places'],
            [ 'ORDER', {'to be confirmed': 'exactly one'}, 'identifying', 'one or more',  'LINE-ITEM', {'label': 'contains'}] 
        ],
        'entity::': {
            'CUSTOMER':  [ 
                ['string', 'name', 'PK', {'comments': 'the name'}], 
                ['string', 'custNumber'] 
            ], 
            'ORDER': [ 
                {'type_att':'int', 'name_att':'orderNumber',  'key_att': 'PK'},
                {'this attribute is not yet valid' : ['string', 'deliveryAdress']}
            ],
            'LINE-ITEM': { 
                'first attribute': ['string', 'productCode',  'PK'],
                'second attribute': ['int',    'quantity'],
                'third attribute': {'type_att':'float',  'name_att':'pricePerUnit'} 
            }   
        },
     } }

diag = MermaidConnec.diagram(order_example2)
display(Image(url="https://mermaid.ink/img/" + b64encode(diag.encode("ascii")).decode("ascii")))

## Example of Flowchart Diagram

### FC-example with simple structure

In [3]:
fc_example = { 
    'json flowchart example:$flowchart' : { 
        'orientation': 'top-down',
        'node::': {
            '0' : ['roundedge', '<b>value</b>'],
            '14': ['roundedge', 'fruits'],
            '13': ['roundedge', '<b>::</b>'],
            '2' : ['rectangle', '<b>kiwi</b>\n<i>3</i>'], 
            '3' : ['rectangle', "<b>mangues</b>\n<i>4</i>"]
        },
        'link::': [ 
            [ '0' , 'normalarrow', '14'],
            [ '14', 'normalarrow', '13'],
            [ '13', 'dottedarrow', '2', 'test'],
            [ '13', 'normalarrow', '3'],         
        ]
     }
}

diag = MermaidConnec.diagram(fc_example)
display(Image(url="https://mermaid.ink/img/" + b64encode(diag.encode("ascii")).decode("ascii")))

### Example of simple JSON 

In [4]:
json_example = { 'example::': {
  "fruits": [
    { "kiwis": 3, "mangues": 4, "pommes": None },
    { "panier": True }
  ],
  "legumes": {
      "patates": "amandine",
      "poireaux": False
   },
   "viandes": ["poisson","poulet","boeuf"]
 }}
Ntv.obj(json_example).to_mermaid('json flowchart', disp=True)

### Example of NTV data

In [5]:
ntv_example = { 'NTV example::': {
  "fruits": [
     {"kiwis": 3, "mangues:int": 4, "pommes": None },
     {"panier": True }
  ],
  "legumes::json": {
      "patates:string": "amandine",
      "poireaux": False
    },
  "viandes": ["poisson",{":string": "poulet"},"boeuf"]
 }}
Ntv.obj(ntv_example).to_mermaid('NTV flowchart', disp=True)

In [6]:
Ntv.obj(ntv_example).to_mermaid('NTV flowchart', disp=True, row=True)

### Example of NTV-TAB data

In [7]:
tab_data   = {'index':           [1, 2, 3],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21'], 
              'value':           [10, 20, 30],
              'value32::int32':  [10, 20, 30],
              'coord::point':    [[1,2], [3,4], [5,6]],
              'names::string':   ['john', 'eric', 'judith']}
Ntv.obj({'tab': tab_data}).to_mermaid('tab data example', disp=True)